# 🕵️ Project: Vision-Based Intruder Detection and Alert System
This notebook sets up a basic vision-based intruder detection system using motion detection and object detection. It can be extended with real-time alerts via email or SMS using APIs like Twilio.

In [ ]:
!pip install opencv-python ultralytics

In [ ]:
import cv2
import time
from ultralytics import YOLO

In [ ]:
# Load YOLOv8 Nano model for fast inference
model = YOLO('yolov8n.pt')

In [ ]:
cap = cv2.VideoCapture(0)
assert cap.isOpened(), 'Could not open webcam'
print("Press 'q' to quit.")

In [ ]:
prev_frame = None
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    if prev_frame is None:
        prev_frame = gray
        continue

    frame_diff = cv2.absdiff(prev_frame, gray)
    thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    motion_detected = any(cv2.contourArea(c) > 5000 for c in contours)
    prev_frame = gray

    if motion_detected:
        print("Motion detected - scanning for intruders...")
        results = model.predict(source=frame, conf=0.5, show=False)
        frame = results[0].plot()

    cv2.imshow('Intruder Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## ✅ Summary
We built a basic intruder detection system using motion detection and YOLO object detection.

You can enhance this system by:
- Sending email/SMS alerts via services like Twilio or SendGrid.
- Saving screenshots when intrusions are detected.
- Deploying on a Raspberry Pi with a camera module.
- Adding facial recognition for known intruders.